In [1]:
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
import rosbag_pandas
import pandas as pd
import matplotlib.pyplot as plt
import easygui
import rosbag
import json
import numpy as np
import cPickle as pkl
import time
import seaborn as sns
import datetime as dt
import fnmatch
import os
import bokeh.palettes as bp
from bokeh.layouts import column
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead, Div
from bokeh.io import export_png
from bokeh.plotting import figure, output_file, show, gridplot,save
from datetime import datetime

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
def pickler( obj, path):
    """
    Pickle a Python object
    """
    with open(path, "wb") as pfile:
        pkl.dump(obj, pfile)

def depickler( path):
    """
    Extracts a pickled Python object and returns it
    """
    with open(path, "rb") as pfile:
        data = pkl.load(pfile)
    return data
import os

def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

# Run the above function and store its results in a variable.   
full_file_paths = get_filepaths("/Users/johnny/Desktop/TEST")

In [2]:

def bokehPlot(df, case, parameters, fig=None,
              TOOLS="pan,crosshair,wheel_zoom\
              ,box_zoom,reset,box_select,lasso_select,undo,redo,save",
              x_range=(506, 526), y_range=(506, 526),
              output_backend="webgl", plot_width=1000, plot_height=1000,
              s=4, xc=513, yc=505,
              xw=20, yw=20, xi=506., yi=503.,
              showPlot=True, title=None, addOdour=True,
              addSmallTit=True, addStart=True, addArrow=True,reallign=False):
    dfc = df[df.trajectory__case == case]
    x = dfc.trajectory__pPos_x
    y = dfc.trajectory__pPos_y
    h = np.deg2rad(dfc.trajectory__pOri_x)

    if 'haw' in parameters['fly']:
        dfcv = dfc.trajectory__valve2

    elif 'apple' in parameters['fly']:
        dfcv = dfc.trajectory__valve1

    ox = dfc[dfcv == True].trajectory__pPos_x
    oy = dfc[dfcv == True].trajectory__pPos_y

    #     opf=dfc[dfc.trajectory__valve1==True].trajectory__pOri_x

    #     import matplotlib as mpl
    #     colors = [
    #         "#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*mpl.cm.viridis(mpl.colors.Normalize()(opf))
    #     ]


    try:
        cm = bp.viridis(max(parameters['odourQuad']) + 1)
        fc = cm[parameters["odourQuad"][case]]
    except TypeError:
        fc = bp.viridis(10)[5]

    if not addSmallTit:
        title = ''

    if fig is None:
        fig = figure(tools=TOOLS, x_range=x_range, y_range=y_range, output_backend=output_backend,
                     plot_width=plot_width, plot_height=plot_height,
                     active_scroll='wheel_zoom', title=title)

    fig.title.align = 'center'
    fig.title.text_font_size = '14pt'

    r = 2
    xs = parameters['playerInitPos'][0]
    ys = parameters['playerInitPos'][1] + 8
    theta = parameters['windQuadOpen'][case] + 180
    if reallign:
        xoffs = xs - 513
        yoffs = ys - 513
    else:
        xoffs=0
        yoffs=0
    # Pos and heading of fly
    fig.triangle(x - xoffs, y - yoffs, size=s, angle=h, fill_alpha=0.5, line_color=None)

    if addOdour:
        # circle at odour pos, with pf encoded in color
        fig.circle(ox, oy, size=2 * s, fill_alpha=0.8, line_color=None, fill_color=fc)

    if addStart:
        # triangle at init pos
        fig.triangle(parameters['playerInitPos'][0], parameters['playerInitPos'][1],
                     size=3 * s, angle=0, fill_alpha=0.9, line_color=None, color='firebrick')

    # arrow

    if parameters["windQuad"][case] == -3:
        lw = 0
        la = 0.1
    elif parameters["windQuad"][case] == -2:
        lw = 4
        la = 1
    else:
        lw = 10
        la = 1
    # print 'le is',lw
    if addArrow:
        fig.add_layout(Arrow(end=VeeHead(size=20), line_color="red", line_alpha=la,

                             x_start=xs, y_start=ys, line_width=lw,
                             x_end=xs + r * np.cos(np.deg2rad(theta)), y_end=ys + r * np.sin(np.deg2rad(theta))))

    return fig

def bokehQuadPlot(df,parameters,path, tr=None,tl=None,bl=None,br=None,
                  TOOLS="pan,crosshair,wheel_zoom,box_zoom,\
                  reset,box_select,lasso_select,undo,redo,save",
                  x_range=(506,526), y_range=(506,526),plot_width=1000, 
                  plot_height=1000,
                 output_backend="webgl",showPlot=False,exposeReturn=False,
                 addBigTit=True,addSmallTit=True, 
                  addArrow=True,addStart=True,addOdour=True):
    
    '''
    plot a single df
    '''

    # output to static HTML file (with CDN resources)
    output_file(path+".html", title="Trajectory Quad", mode="cdn")


    # create a new plot with the tools above, and explicit ranges
    w=500
    h=500 
    s=6
    xc=516
    yc=513
    xw=20
    yw=20
    xi=xc-xw/2.
    yi=yc-yw/2.

    
    
    ls=parameters['loadingString']
    wq=parameters['windQuad']
    oq=parameters['odourQuad']
    wqo=parameters['windQuadOpen']
    
    def quadTitGen(quad,ls=ls,wq=wq,oq=oq,wqo=wqo):
        tit='odour : '+str(oq[quad])+'\t wind : '+str(wq[quad])+'\t windDir : '+str(wqo[quad])
#         tit='ls : '+str(ls)+'\t odour : '+str(oq[quad])+'\t wind : '+str(wq[quad])+'\t windDir : '+str(wqo[quad])

        return tit
    
    
    bl=bokehPlot(df,2,parameters,TOOLS=TOOLS,title=quadTitGen(2),
                 fig=bl,addSmallTit=addSmallTit,addStart=addStart,
                 addArrow=addArrow,addOdour=addOdour
                 ,x_range=x_range, y_range=y_range)
    tr=bokehPlot(df,0,parameters,x_range=bl.x_range, y_range=bl.y_range,TOOLS=TOOLS,title=quadTitGen(0),
                 fig=tr,addSmallTit=addSmallTit,addStart=addStart,
                 addArrow=addArrow,addOdour=addOdour)
    tl=bokehPlot(df,1,parameters,x_range=bl.x_range, y_range=bl.y_range,TOOLS=TOOLS,title=quadTitGen(1),
                 fig=tl,addSmallTit=addSmallTit,addStart=addStart
                 ,addArrow=addArrow,addOdour=addOdour)
    br=bokehPlot(df,3,parameters,x_range=bl.x_range, y_range=bl.y_range,TOOLS=TOOLS,title=quadTitGen(3),
                 fig=br,addSmallTit=addSmallTit,addStart=addStart,
                 addArrow=addArrow,addOdour=addOdour)


    p = gridplot([[tl, tr], [bl, br]])
#     export_png(p, filename=path+".png")
    if addBigTit:
        p=column(Div(text=path.split('/')[-1]),p)
    
    if exposeReturn:#if you want access to all plots
        return p,tr,tl,bl,br
    else:
            
        if showPlot:
            show(p)
        else:
            save(p)

        return p,None,None,None,None


def pickle2df(path=None):
    '''Load a dataframe from pickle'''
    defaultPath="/home/rhagoletis/catkin/src/World/bags/"
    if path is None:
        path=easygui.fileopenbox(title="Bags to Dataframes"
                                  ,default=defaultPath,
                                  multiple=False,filetypes=["*df.pickle"])
        
#     print path, "\n"
#     for path in paths:
#     print "\n\ncurrentl;y analysisnh",path
    # picklepath=path+"_df.pickle"
    data=depickler(path)
    df=data["df"]
    parameters=data['metadata']['parameters']

    return df,parameters,path




def pickle2bokeh(path=None,showPlot=True,exposeReturn=False,tr=None,tl=None,bl=None,br=None,
                 addBigTit=True,addSmallTit=True,addArrow=True,addStart=True,addOdour=False,
                 x_range=(506,526), y_range=(506,526)):
    
    df,parameters,path=pickle2df(path=path)
    try:
        plot,tr,tl,bl,br=bokehQuadPlot(df=df,parameters=parameters,path=path,showPlot=showPlot,
                                       exposeReturn=exposeReturn,tr=tr,tl=tl,bl=bl,br=br,
                                       addBigTit=addBigTit,addSmallTit=addSmallTit, addArrow=addArrow,
                                       addStart=addStart,addOdour=addOdour,
                                       x_range=x_range, y_range=y_range)
    except SyntaxError as e:
        print "\n\n\n Exception",e
        return None,None,None,None
    
    if exposeReturn:#if you want access to all plots
        return plot,df,parameters,path,tr,tl,bl,br
    else:
        return plot,df,parameters,path


def fileList(foldPath,included=['*bag_df.pickle'],excluded=[]):

    
    matches = []
    for root, dirnames, filenames in os.walk(foldPath):
        
        for include in included:
            for filename in fnmatch.filter(filenames, include):
                toKeep=True
                
                for exclude in excluded:
                    if exclude in filename:
                        toKeep=False
                    
                if toKeep:
                    matches.append(os.path.join(root, filename))
    matches.sort()
    return matches


def pickleDir2bokeh(foldPath=None,showPlot=False,
                    addBigTit=True,addSmallTit=True,addArrow=True,addStart=True,
                    x_range=(506, 526), y_range=(506, 526),
                   defaultPath='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/agg/odour+wind/',):
    tstart = datetime.now()

    if foldPath is None:
        foldPath= easygui.diropenbox(    default=defaultPath)
    
    matches=fileList(foldPath,included=['*bag_df.pickle'],excluded=['pf'])


    matches.sort(reverse=True)
    print matches
    
    i=1 #file progress index
    for path in matches:
        
        tic()
        plot, df,parameters,path =pickle2bokeh(path,showPlot=showPlot,
                          addBigTit=addBigTit,addSmallTit=addSmallTit,
                           addArrow=addArrow,addStart=addStart,
                          x_range=x_range, y_range=y_range,)

#         df,parameters=pickle2df(path)
        
#         try:
#             bokehQuadPlot(df=df,parameters=parameters,path=path)
#         except Exception as e:
#             print "\n\n\n Exception",e
#             pass
        
        print('\n\n Currently anal on {}. \n It is {} / {} file'.format(path.split('/')[-1], i,len(matches)))

        i+=1
        toc()
    
    tend = datetime.now()
    print "Total time",tend - tstart

    

# def dfDir2bokeh():
    
#     from datetime import datetime

#     tstart = datetime.now()



#     foldPath= easygui.diropenbox(    default="/home/rhagoletis/catkin/src/World/bags/")
#     matches = []
#     for root, dirnames, filenames in os.walk(foldPath):
#         for filename in fnmatch.filter(filenames, '*traj.bag_df.pickle'):
#             if 'pf' in filename:
#                 continue
#             matches.append(os.path.join(root, filename))
#     matches.sort(reverse=True)
#     print matches
    
#     i=1
#     for path in matches:
        
#         tic()
        
# #         df,parameters=pickle2bokeh(path)
#         df,parameters,path2=pickle2df(path)
        
#         try:
#             bokehQuadPlot(df=df,parameters=parameters,path=path,showPlot=False)
#         except Exception as e:
#             print "\n\n\n Exception",e
#             pass
        
#         print('\n\n Currently anal on {}. \n It is {} / {} file'.format(path.split('/')[-1], i,len(matches)))

# #         print "Currently analysing", path
#         i+=1
#         toc()
    
#     tend = datetime.now()
#     print "Total time",tend - tstart


def bag2pickle(paths=None,defaultPath="/home/rhagoletis/catkin/src/World/bags/"):
    '''
    Load  bag filesto make into respective dataframes
    '''
    # defaultPath="/home/rhagoletis/catkin/src/World/bags/"
    if paths is None:
        paths=easygui.fileopenbox(title="Bags to Dataframes"
                                  ,default=defaultPath,
                                  multiple=True,filetypes=["*traj.bag"])
    
    print paths, "\n"
    metadata=None
    tic()
    i=1
    for path in paths:
        tic()
        print "starting analysis of file %s , %s / %s files" %(path.split('/')[-1],i,len(paths))
        try:
            df=rosbag_pandas.bag_to_dataframe(path,include=['/trajectory'])
        except Exception as e:
            print "Bag has some nonetype error",path , e 
            df=None
            parameters=None
            picklepath = None
            continue
        bag=rosbag.Bag(path)

        try:
            for topic,msg,t in bag.read_messages(topics='/metadata'):
                a=msg
    #         parameters=json.loads(a.data)
    #         metadata={"meta":parameters}

            metadata=json.loads(a.data)
            parameters=metadata['parameters']

        except :
            print "no such file!, trying the pickle"
            try:
                metadata=depickler(paths[0].split('.bag')[0])
            except IOError:
                metadata=depickler(paths[0].split('.bag')[0]+'.pickle')

            parameters=metadata['parameters']


        obj=dict(df=df,metadata=metadata)
        
        picklepath=path+"_df.pickle"
        pickler(obj,picklepath)
    #     df.to_pickle(picklepath)

        i+=1
        toc()
    print "\nanalysis of %s files complete" %len(paths)
    return df, parameters,picklepath
    
def bagDir2pickle(foldPath=None,defaultPath="/home/rhagoletis/catkin/src/World/bags/",postProcFunc=None,
                  included=['*.bag'],excluded=[]):
    tstart = datetime.now()
    
    if foldPath is None:
        foldPath= easygui.diropenbox(default=defaultPath)
    
    matches=fileList(foldPath,included=included,excluded=excluded)
#     matches = []
#     for root, dirnames, filenames in os.walk(foldPath):
#         for filename in fnmatch.filter(filenames, '*traj.bag_df.pickle'):
#             if 'pf' in filename:
#                 continue
#             matches.append(os.path.join(root, filename))

    matches.sort(reverse=True)
    print matches
    
    i=1 #file progress index
    for path in matches:
        
        try:
            tic()
            df, parameters,picklepath=bag2pickle(paths=[path])
            if df is None:
                continue

            if postProcFunc is not None:
                postProcFunc(df,parameters,picklepath)

            print('\n\n Currently anal on {}. \n It is {} / {} file'.format(path.split('/')[-1], i,len(matches)))

            i+=1
            toc()
        except Exception as e:
            print "something odd in ",path, e

    tend = datetime.now()
    print "Total time",tend - tstart

def bagDir2bokeh(foldPath=None,defaultPath="/home/rhagoletis/catkin/src/World/bags/",showPlot=False,excluded=['full']):
    bagDir2pickle(foldPath,defaultPath=defaultPath,postProcFunc=bokehQuadPlot,excluded=excluded)
# def xDir2y(x2y,included=['*.bag'],excluded=[],defaultPath="/home/rhagoletis/catkin/src/World/bags/"):
    
#     tstart = datetime.now()

#     foldPath= easygui.diropenbox(default=defaultPath)
    
#     matches=fileList(foldPath,included=included,excluded=excluded)
# #     matches = []
# #     for root, dirnames, filenames in os.walk(foldPath):
# #         for filename in fnmatch.filter(filenames, '*traj.bag_df.pickle'):
# #             if 'pf' in filename:
# #                 continue
# #             matches.append(os.path.join(root, filename))

#     matches.sort(reverse=True)
#     print matches
    
#     i=1 #file progress index
#     for path in matches:
        
#         tic()
#         x2y(paths=[path])
        
#         print('\n\n Currently anal on {}. \n It is {} / {} file'.format(path.split('/')[-1], i,len(matches)))

#         i+=1
#         toc()
    
#     tend = datetime.now()
#     print "Total time",tend - tstart

# bag2pickle()
# bagDir2pickle()
# pickle2bokeh()
# pickle2df()
# dfDir2bokeh()


In [ ]:
def plotAggreg(foldPath=None,
               defaultPath='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/agg/odour+wind/',
              x_range=(493,533), y_range=(485,525),):
    if foldPath is None:
        foldPath=easygui.diropenbox(default=defaultPath)
    files=fileList(foldPath,included=['*.bag_df.pickle'])
    tr=None
    tl=None
    bl=None
    br=None
    iTot=len(files)
    i=1
    for item in files:
        plot,df,parameters,path,tr,tl,bl,br=pickle2bokeh(item,exposeReturn=True,tr=tr,tl=tl,bl=bl,br=br,
                     addBigTit=False,addSmallTit=False,addArrow=False,addStart=False,
                        x_range=x_range, y_range=y_range,)
        print ("currently analysing {}, {}/{} files ").format(item.strip('/')[-1],i,iTot)
        i+=1
    show(plot)
    return plot

In [ ]:
df=Out[56]
df=df[0]

In [ ]:
xc=514
yc=513
w=19
xw=w
yw=w
xr=(xc-xw,xc+xw)
yr=(yc-yw,yc+yw)

# pickleDir2bokeh(addBigTit=False,addSmallTit=False,addArrow=False,
#                 addStart=False,x_range=xr,y_range=yr,
#                 defaultPath='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/agg/odour+wind/')
out=plotAggreg(x_range=xr,y_range=yr)

In [3]:
a=pickle2df()

In [3]:
df=a[0]
reset=np.where(df["trajectory__reset"] == True)[0]
print reset

NameError: name 'a' is not defined

In [10]:
import arrow

In [64]:
import easygui
import arrow

bagPath='/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_full.bag'
runNumStart=12
caseStart=2

# bagPath=easygui.fileopenbox(filetypes=['*.bag'])
# runNumStart=easygui.integerbox(msg='wnter the start runNum')
# caseStart=easygui.integerbox(msg='enter the start case')

# runNumStop=easygui.integerbox(msg='wnter the stop runNum')
# caseStop=easygui.integerbox(msg='enter the start case')

pklP='/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_full.bag_df.pickle'
df,parameters,path=pickle2df(path=pklP)


In [70]:
df.trajectory__DCoffset[100:103]

2017-09-03 12:17:00.432092928    0.0
2017-09-03 12:17:00.438046976    0.0
2017-09-03 12:17:00.444192000    0.0
Name: trajectory__DCoffset, dtype: float64

In [71]:
plt.scatter(x=df.trajectory__pPos_x[232743:],y=df.trajectory__pPos_y[232743:])
plt.show()

In [50]:
reset=np.where(df["trajectory__reset"] == True)[0]
reset

array([     0,   4951,   9903,  14855,  19807,  24759,  29711,  34663,
        39615,  44567,  49519,  54471,  59423,  64375,  69327,  74279,
        79231,  84183,  89135,  94087,  99039, 103991, 108943, 113895,
       118847, 123799, 128751, 133703, 138655, 143607, 148559, 153511,
       158463, 163415, 168367, 173319, 178271, 183223, 188175, 193127,
       198079, 203031, 207983, 212935, 217887, 222839, 227791, 232743])

In [51]:
startI=np.where((df["trajectory__runNum"]==runNumStart)&(df["trajectory__case"]==caseStart))[0][0]
print startI
dt=df.index[startI]

232743


In [107]:
df.ix[2:,:]

,trajectory__DCoffset,trajectory__boutFrame,trajectory__case,trajectory__compensation,trajectory__gain,trajectory__groundSpeed,trajectory__headingControl,trajectory__impose,trajectory__imposeHeading,trajectory__imposeResponse,...,trajectory__servoAngle,trajectory__slip,trajectory__speed,trajectory__speedControl,trajectory__trial,trajectory__valve1,trajectory__valve2,trajectory__valve3,trajectory__wbad,trajectory__wbas
2017-09-03 12:16:59.837987840,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,2.257990e-02,1.929611
2017-09-03 12:16:59.844110080,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,2.257990e-02,1.929611
2017-09-03 12:16:59.850136064,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,2.257990e-02,1.929611
2017-09-03 12:16:59.856168960,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,4.515979e-02,1.907031
2017-09-03 12:16:59.862255104,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,4.515979e-02,1.907031
2017-09-03 12:16:59.868352000,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,2.257990e-02,1.974771
2017-09-03 12:16:59.874411008,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,2.257990e-02,1.974771
2017-09-03 12:16:59.880434944,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.000000e+00,1.952191
2017-09-03 12:16:59.886446848,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,2.257990e-02,1.974771
2017-09-03 12:16:59.892507904,0.0,0.0,0.0,0.0,16.0,0.000000,0.0,0.0,0.0,0.0,...,270.0,270.000000,0.0,0.0,1.0,0.0,0.0,0.0,2.257990e-02,1.974771


In [109]:
save(bokehPlot(df,3,parameters,title=''),path+'_3TrajOver.html')
save(bokehPlot(df,3,parameters,title='',addOdour=False),path+'_3Traj.html')
save(bokehPlot(df.ix[startI:,:],3,parameters,title=''),path+'_3SelTrajOver.html')
save(bokehPlot(df.ix[startI:,:],3,parameters,title='',addOdour=False),path+'_3SelTraj.html')

# startI

'/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_full.bag_df.pickle_3SelTraj.html'

In [111]:
import bokehplot

ImportError: No module named bokehplot

In [84]:
import easygui
import arrow
# import bokehPlot
import numpy as np
import subprocess

bagPath='/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_traj.bag'
runNumStart=12
caseStart=2

# bagPath=easygui.fileopenbox(filetypes=['*.bag'])
# runNumStart=easygui.integerbox(msg='wnter the start runNum')
# caseStart=easygui.integerbox(msg='enter the start case')
outBag='out.bag'
# runNumStop=easygui.integerbox(msg='wnter the stop runNum')
# caseStop=easygui.integerbox(msg='enter the start case')


df,parameters,path=bag2pickle([bagPath])
startI=np.where((df["trajectory__runNum"]==runNumStart)&(df["trajectory__case"]==caseStart))[0][0]
print startI
dt=df.index[startI]
a=arrow.get(dt)
ts=a.timestamp
ts=ts+a.microsecond


filt = subprocess.Popen(['rosbag','filter',bagPath,outBag,'"t.to_sec()>='+str(ts)+'"'], stdout=subprocess.PIPE)

['/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_full.bag'] 

starting analysis of file 2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_full.bag , 1 / 1 files
Elapsed time: 100.137490 seconds.


analysis of 1 files complete
227791


In [93]:
bagPath='/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_traj.bag'
runNumStart=12
caseStart=3
dt=df.index[startI]
a=arrow.get(dt)
ts=a.timestamp
ts=ts+(a.microsecond*.00001)


filt = subprocess.Popen(['rosbag','filter',bagPath,outBag,'"t.to_sec()>='+str(ts)+'"'], stdout=subprocess.PIPE)

In [94]:
filt.communicate()

('\r out.bag                                       0%   0.0 KB / 107.1 MB --:-- ETA\r out.bag                                       2%   2.3 MB / 107.1 MB 00:46 ETA\r out.bag                                       4%   4.5 MB / 107.1 MB 00:45 ETA\r out.bag                                       6%   6.8 MB / 107.1 MB 00:44 ETA\r out.bag                                       8%   9.0 MB / 107.1 MB 00:43 ETA\r out.bag                                      10%  10.8 MB / 107.1 MB 00:44 ETA\r out.bag                                      11%  12.8 MB / 107.1 MB 00:44 ETA\r out.bag                                      13%  14.8 MB / 107.1 MB 00:43 ETA\r out.bag                                      15%  17.0 MB / 107.1 MB 00:42 ETA\r out.bag                                      17%  19.1 MB / 107.1 MB 00:41 ETA\r out.bag                                      19%  21.2 MB / 107.1 MB 00:40 ETA\r out.bag                                      21%  23.3 MB / 107.1 MB 00:39 ETA\r out.bag                

In [63]:
a=arrow.get(dt)
a.timestamp

1504442430

In [37]:
arrow.get(arrow.get(dt).timestamp)

<Arrow [2017-09-03T12:40:30+00:00]>

In [5]:
dt2=arrow.get(dt).timestamp
print dt2
arrow.get(dt2)

TypeError: Can't parse single argument type of '<type 'module'>'

In [6]:
arrow.utcnow()

<Arrow [2017-09-17T11:21:05.272896+00:00]>

In [7]:
a[1]["bagFileName"]

NameError: name 'a' is not defined

In [8]:
df.index[0]

NameError: name 'df' is not defined

In [9]:
arrow.get(df.index[0])

NameError: name 'df' is not defined

In [32]:
arrow.get(df.index[0]).to('+05:30')



<Arrow [2017-09-03T17:46:59.829664+05:30]>

In [33]:
(arrow.get(df.index[0]).to('+05:30')).timestamp

1504441019

In [35]:
(arrow.get(df.index[0])).timestamp

1504441019

In [21]:
dt=df.index[232743]

In [ ]:
df=Out[115][0]

In [ ]:
5>2>1

In [ ]:
((df["trajectory__pPos_x"]>520)).plot()
plt.show()

In [ ]:
((df["trajectory__pPos_x"] > 513.2) & (df["trajectory__pPos_x"] < 513.4) & (df["trajectory__pPos_y"] > 525.8) & (df["trajectory__pPos_y"] < 526))
# plt.show()

In [ ]:
len(df)

In [ ]:
a=np.where(((513.2<df["trajectory__pPos_x"])&(513.4>df["trajectory__pPos_x"])&(525.8<df["trajectory__pPos_y"])&(526>df["trajectory__pPos_y"])))[0]

In [ ]:
list(np.where(df["trajectory__reset"] == True))

In [7]:
reset=np.where(df["trajectory__reset"] == True)[0]
print reset
pos=241429
reset[np.argmin((pos-reset)*((pos-reset)>0)*1)-1]

[     0   4951   9903  14855  19807  24759  29711  34663  39615  44567
  49519  54471  59423  64375  69327  74279  79231  84183  89135  94087
  99039 103991 108943 113895 118847 123799 128751 133703 138655 143607
 148559 153511 158463 163415 168367 173319 178271 183223 188175 193127
 198079 203031 207983 212935 217887 222839 227791 232743]


227791

In [ ]:
(pos-reset)*((pos-reset)>0)*1

In [39]:
dfnow = df.ix[200, :]

In [42]:
from PyKDE4.kdeui import *

In [40]:
dfnow

trajectory__DCoffset                       0
trajectory__boutFrame                      0
trajectory__case                           0
trajectory__compensation                   0
trajectory__gain                          16
trajectory__groundSpeed               1.0033
trajectory__headingControl                 1
trajectory__impose                         0
trajectory__imposeHeading                  0
trajectory__imposeResponse                 0
trajectory__imposeResponseHeading          0
trajectory__imposeResponseSmooth           0
trajectory__isFlying                       1
trajectory__key                             
trajectory__o1Pos_x                        0
trajectory__o1Pos_y                        0
trajectory__o1Pos_z                        0
trajectory__o2Pos_x                        0
trajectory__o2Pos_y                        0
trajectory__o2Pos_z                        0
trajectory__pOri_x                   17.7026
trajectory__pOri_y                         0
trajectory

In [ ]:
a

In [ ]:
plt.plot(a)
plt.show()

In [ ]:
plt.plot((np.where(((513.05<df["trajectory__pPos_x"])<513.1))))
plt.show()

In [ ]:
a

In [ ]:
# np.where(df["trajectory__reset"] == True)
list(np.where(df["trajectory__reset"] == True)[0])

In [ ]:
df.ix[df.index[df.trajectory__reset==True].tolist()[0]]
# df.index[df['BoolCol']].tolist()

In [ ]:
pickleDir2bokeh(addBigTit=False,addSmallTit=False,addArrow=False,addStart=False)

In [ ]:
bagDir2bokeh(defaultPath='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31')

In [ ]:
from bokeh.io import export_svgs
plot=Out[21]
plot.output_backend = "svg"
export_svgs(plot, filename="plot.svg")

In [ ]:
dfDir2bokeh()

In [ ]:
easygui.fileopenbox()

In [ ]:
bagDir2pickle(defaultPath=)

In [ ]:
p,df,parameters,path=pickle2bokeh()

In [ ]:
show(bokehPlot(df,2,parameters,plot_width=1280,plot_height=1280))

In [ ]:


plot,df,parameters,path=pickle2bokeh()
# bokehQuadPlot(df,parameters,path)


# dfDir2bokeh()


In [ ]:
showPlot(bokehPlot(df,2,parameters,plot_width=1280,plot_height=1280))